In [10]:
#from flashtext import KeywordProcessor
from mstr_robotics.navigation import answer_prompts, mstr_objects
from mstr_robotics.report import rep, prompts
from mstrio.connection import Connection
from IPython.display import HTML
import pandas as pd

import json 
import os


from mstr_robotics.user_RAG import keyword_processor, perplexity

u_keyword_processor=keyword_processor()
os_mcp_folder_str="C:\\coding\\python_io\\output_files\\MCP_data"

user_path="..\\config\\user_d.json"
with open(user_path, 'r') as file:
    user_d = json.load(file)

i_prompts=prompts()
i_rep=rep()
u_perplexity=perplexity()
i_mstr_objects=mstr_objects()

In [11]:
project_id = "B7CA92F04B9FAE8D941C3E9B7E0CD754"
conn_params =  user_d["conn_params"]
conn_params["project_id"]=project_id
conn = Connection(**conn_params)
conn.headers['Content-type'] = "application/json"

Connection to Strategy One Intelligence Server has been established.


In [12]:
# MSTR specitic objects and definitions
attribute_form_elements_df = pd.read_csv(os.path.join(os_mcp_folder_str, "attribute_form_elements.csv"))
attribute_elements_df = pd.read_csv(os.path.join(os_mcp_folder_str, "attribute_elements.csv"))
att_form_def_df = pd.read_csv(os.path.join(os_mcp_folder_str, "att_form_def.csv"))
obj_prp_rel_df = pd.read_csv(os.path.join(os_mcp_folder_str, "obj_prp_rel.csv"))
dos_rep_prp_rel_df = pd.read_csv(os.path.join(os_mcp_folder_str, "dos_rep_prp_rel.csv"))
dashboard_definitions_df = pd.read_csv(os.path.join(os_mcp_folder_str, "dashboard_definitions.csv"))
dashboard_chapter_filter_df = pd.read_csv(os.path.join(os_mcp_folder_str, "dashboard_chapter_filter.csv"))
dashboard_selector_filter_df = pd.read_csv(os.path.join(os_mcp_folder_str, "dashboard_selector_filter.csv"))

i_answer_prompts=answer_prompts(attribute_form_elements_df=attribute_form_elements_df
                                , attribute_elements_df=attribute_elements_df
                                , obj_prp_rel_df=obj_prp_rel_df
                                , att_form_def_df=att_form_def_df
                                , dos_rep_prp_rel_df=dos_rep_prp_rel_df
                                , dashboard_definitions_df=dashboard_definitions_df
                                , dashboard_chapter_filter_df=dashboard_chapter_filter_df
                                , dashboard_selector_filter_df=dashboard_selector_filter_df
                                )

In [13]:
# tool agnostic definitions. Only MSTR for the moment
element_df_d_l=[{"df":attribute_form_elements_df,"key_col":"element_val","key_type":"element_val","rag_cols": ["attribute_name", "form_name", "element_val"]},
                 {"df":attribute_elements_df,"key_col":"element_val","key_type":"element_val","rag_cols": ["attribute_name", "element_val"]}
                ]

bi_obj_df=obj_prp_rel_df[["object_name", "obj_type", "object_id"]][obj_prp_rel_df["obj_type"].isin(["attribute","metric"]) ]

obj_df_d_l=[{"df":bi_obj_df,"key_col":"object_name","key_type":"object_name","rag_cols": ["object_name", "obj_type"]}]
            


In [15]:
msg_t="Please show me the Cost_1, Revenue and Profit for the attributes Year,Category, Region"
msg_t= msg_t + " and filter for the yaer 2021, 2022 and 2023"
msg_t= msg_t + " and Categories starting with B"
msg_t= msg_t + " and Revenue is between 10 and 1000000"


key_word_l=u_keyword_processor.extract_keywords(msg_t=msg_t)

att_elem_str=i_mstr_objects.get_att_elem_str(element_df_d_l, key_word_l=key_word_l)
bi_obj_str=i_mstr_objects.get_att_elem_str(obj_df_d_l, key_word_l=key_word_l)

sys_cont=u_perplexity.rag_sys_cont(key_word_l=key_word_l,att_elem_str=att_elem_str, bi_obj_str=bi_obj_str)

message_check_d={}
message_check_d_l=[]
message_check_d["msg_nr"] = "1"
message_check_d["msg_t"] = msg_t
message_check_d=u_perplexity.call_perplexity( msg_t=msg_t, sys_cont=sys_cont, message_check_d=message_check_d, temperature=0.1)
message_check_d_l.append(message_check_d.copy())

#message_check_d["step"] = "template"
#message_check_d = u_perplexity.call_perplexity(msg_t=msg_t,message_check_d=message_check_d, sys_cont=sys_cont)
#message_check_d_l.append(message_check_d.copy())
bi_request_d=u_perplexity.parse_and_structure(message_check_d_l)
bi_request_d

{'msg_nr': '1',
 'msg_t': 'Please show me the Cost_1, Revenue and Profit for the attributes Year,Category, Region and filter for the yaer 2021, 2022 and 2023 and Categories starting with B and Revenue is between 10 and 1000000',
 'attributes': ['Year', 'Category', 'Region'],
 'metrics': ['Cost', 'Revenue', 'Profit'],
 'filter': {'att_element_1': {'attribute': 'Year',
   'operator': 'In',
   'element_list': ['2021', '2022', '2023']},
  'att_qual_1': {'attribute': 'Category',
   'Column': 'desc',
   'operator': 'BeginsWith',
   'value': 'B'},
  'metric_filter_1': {'level': None,
   'metric': 'Revenue',
   'operator': 'Between',
   'value': [10, 1000000]}},
 'question': ''}

In [18]:
report_id="25D40AD444B6D51B333021ADFB219501"

prompt_answ=i_answer_prompts.AI_mstr_prp_page_ans( vector_store=u_keyword_processor
                                                  ,bi_request_d=bi_request_d
                                                  ,rep_dos_id=report_id
                                                  )
prompt_answ


att_elemen
att_qual_1
metric_fil


'{"prompts": [{"id": "1F21042E4FC08886A9D7E4920CA4EC59", "type": "EXPRESSION", "answers": {"expression": {"operator": "And", "operands": [{"operator": "AND", "operands": [{"operator": "In", "operands": [{"type": "form", "attribute": {"id": "8D679D5111D3E4981000E787EC6DE8A4"}, "form": {"id": "45C11FA478E745FEA08D781CEA190FE5"}}, {"type": "constants", "dataType": "Numeric", "values": ["2021", "2022", "2023"]}]}, {"operator": "BeginsWith", "operands": [{"type": "form", "attribute": {"id": "8D679D3711D3E4981000E787EC6DE8A4"}, "form": {"id": "CCFBE2A5EADB4F50941FB879CCF1721C"}}, {"type": "constant", "dataType": "Char", "value": "B"}]}]}]}}}, {"id": "14B57F1F46127BC6E6F746888BC3C7CB", "type": "EXPRESSION", "answers": {"expression": {"operator": "AND", "operands": [{"operator": "Between", "operands": [{"type": "metric", "id": "4C05177011D3E877C000B3B2D86C964F"}, {"type": "constant", "dataType": "Real", "value": "10"}, {"type": "constant", "dataType": "Real", "value": "1000000"}], "level": {"t

In [17]:
ai_rep_name="dyn_prompt_page_botstat"
ai_rep_folder_id="2F2302AE4D1C2DDDFA9CDCB46802B185"
report_id="25D40AD444B6D51B333021ADFB219501"

rep_id=i_answer_prompts.save_AI_rep(conn=conn,report_id=report_id
                                  ,prompt_answ=prompt_answ
                                  ,ai_rep_name=ai_rep_name
                                  ,ai_rep_folder_id=ai_rep_folder_id)
new_rep_id=rep_id.json()["id"]
instance_id = i_rep.open_Instance(conn=conn, report_id=new_rep_id)
df=i_rep.report_df(conn=conn, report_id=new_rep_id, instance_id=instance_id)
df

Report object named: 'dyn_prompt_page_botstat' with ID: 'C731B12148180FDA527FB5BCE95E618E'


,Year,Category,Region,Cost,Revenue,Profit
0,2021,Books,Central,97277.440,124045.80,26768.360
1,2021,Books,Mid-Atlantic,89965.200,114815.95,24850.750
2,2021,Books,Northeast,171114.632,218225.70,47111.068
3,2021,Books,Northwest,35559.875,45521.65,9961.775
4,2021,Books,South,105501.656,133762.05,28260.394
5,2021,Books,Southeast,44394.129,56494.00,12099.871
6,2021,Books,Southwest,74955.595,95810.20,20854.605
7,2021,Books,Web,62410.790,79531.15,17120.360
8,2022,Books,Central,121215.783,154588.50,33372.717
9,2022,Books,Mid-Atlantic,107270.150,136809.85,29539.700


In [19]:
ai_rep_name="dyn_prompt_page_botstat"
report_id="25D40AD444B6D51B333021ADFB219501"
#report_id="29FBB4884282ED7203AFD2B247C57688"

rep_id=i_answer_prompts.save_AI_rep(conn=conn,report_id=report_id
                                  ,prompt_answ=prompt_answ
                                  ,ai_rep_name=ai_rep_name
                                  ,promptOption ="filterAndTemplate"
                                  ,ai_rep_folder_id=ai_rep_folder_id)
new_rep_id=rep_id.json()["id"]
link=i_rep.web_base_url(conn=conn,report_id=new_rep_id)
HTML(link)

http://217.154.213.84:8080/MicroStrategy/servlet/mstrWeb?Server=217.154.213.84&Project=MicroStrategy+Tutorial&evt=4001&src=mstrWeb.4001&reportViewMode=1&reportID=C731B12148180FDA527FB5BCE95E618E&currentViewMedia=2


In [8]:
f="C:\\Users\danie\\Downloads\\display_difference_dash.xlsx"
df = pd.read_excel(f)
result = df.groupby('org_obj_key')['json_key_path'].apply(list).reset_index()
d_l=result.to_dict


,org_obj_key,json_key_path
0,ATTRIBUTE:8D679D3511D3E4981000E787EC6DE8A4,"[displays.browseDisplays[1], displays.reportDisplays[1]]"
1,DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A,"[chapters[0].pages[0].selectors[0].currentSelection.elements[0], chapters[0].pages[0].selectors[..."
2,METRIC:4C051DB611D3E877C000B3B2D86C964F,[smartTotal]
3,METRIC:7FD5B69611D5AC76C000D98A4CC5F24F,"[expression.text, expression.tokens[10].type, expression.tokens[10].value, expression.tokens[11]..."
4,REPORT_DEFINITION:C060EF484E1352B26D945CBAF191F2B8,"[advancedProperties.drillOptions.drillingEnableReportDrilling.isInherited, advancedProperties.dr..."


In [12]:
d_l=result.to_dict('records')
d_l

[{'org_obj_key': 'ATTRIBUTE:8D679D3511D3E4981000E787EC6DE8A4',
  'json_key_path': ['displays.browseDisplays[1]',
   'displays.reportDisplays[1]']},
 {'org_obj_key': 'DOCUMENT_DEFINITION:98EB31B54122FFB738E6E08A2F29421A',
  'json_key_path': ['chapters[0].pages[0].selectors[0].currentSelection.elements[0]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[1]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[10]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[11]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[12]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[13]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[14]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[15]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[16]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[17]',
   'chapters[0].pages[0].selectors[0].currentSelection.elements[